# PyTorch lightning project template 

In [1]:
import os
import pytorch_lightning as pl
from argparse import ArgumentParser
from pytorch_lightning import Trainer
import pytorch_lightning.callbacks as plc
from pytorch_lightning.loggers import TensorBoardLogger
from model import MInterface
from data import DInterface
from torch.utils.data import DataLoader,random_split
import importlib
from utils import load_model_path_by_args,load_callbacks

C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Hyperparameters Setting

In [11]:
parser = ArgumentParser()
# Basic Training Control
parser.add_argument('--train_datatype', default='hdf5', type=str)
parser.add_argument('--train_h5path', default='./dataset/train_30.hdf5', type=str)
parser.add_argument('--test_h5path', default=None, type=str)
parser.add_argument('--image_dir', default='./dataset/image/', type=str)
parser.add_argument('--msk_dir', default='./dataset/mask/', type=str)
parser.add_argument('--test_img_dir', default=None, type=str)
parser.add_argument('--test_msk_dir', default=None, type=str)
parser.add_argument('--split_rate', default=0.2, type=int)
parser.add_argument('--aug', default='true', type=str)
parser.add_argument('--stage', default='train', type=str)


# Basic Training Control
parser.add_argument('--input_size', default=256, type=int)
parser.add_argument('--batch_size', default=60, type=int)
parser.add_argument('--num_workers', default=4, type=int)
parser.add_argument('--seed', default=42, type=int)
parser.add_argument('--lr', default=1e-3, type=float)

# LR Scheduler
parser.add_argument('--lr_scheduler', choices='cosine', type=str)
parser.add_argument('--lr_decay_steps', default=20, type=int)
parser.add_argument('--lr_decay_rate', default=0.5, type=float)
parser.add_argument('--lr_decay_min_lr', default=1e-5, type=float)

# Restart Control
parser.add_argument('--load_best', action='store_true')
parser.add_argument('--load_dir', default=None, type=str)
parser.add_argument('--load_ver', default=None, type=str)
parser.add_argument('--load_v_num', default=None, type=int)

# Training Info
parser.add_argument('--dataset', default='Segh5_data', type=str)
parser.add_argument('--data_dir', default='ref/data', type=str)
parser.add_argument('--loss', default='bce', type=str)
parser.add_argument('--weight_decay', default=1e-5, type=float)
parser.add_argument('--no_augment', action='store_true')
parser.add_argument('--log_dir', default='lightning_logs', type=str)

# Model Hyperparameters
parser.add_argument('--model_name', default='unet_vgg', type=str)
parser.add_argument('--encoder_name', default='vgg16_bn', type=str)
parser.add_argument('--encoder_weights', default=None, type=str)
parser.add_argument('--activation', default=None, type=str)
parser.add_argument('--encoder_depth', default=5, type=int)
parser.add_argument('--class', default=2, type=int)

# Other
parser.add_argument('--aug_prob', default=0.5, type=float)

# Add pytorch lightning's args to parser as a group.
parser = Trainer.add_argparse_args(parser)

## Deprecated, old version
# parser = Trainer.add_argparse_args(
#     parser.add_argument_group(title="pl.Trainer args"))

# Reset Some Default Trainer Arguments' Default Values
parser.set_defaults(progress_bar_refresh_rate=5)
parser.set_defaults(max_epochs=10)

args = parser.parse_args([])

## Training

In [12]:
def load_data_module(name,**kwargs):
    # Please always name your model file name as `snake_case.py` and
    # class name corresponding `CamelCase`.
    camel_name = ''.join([i for i in name.split('_')])
    try:
        data_module = getattr(importlib.import_module('data.'+name, package=__package__), camel_name)
    except:
        raise ValueError(
            f'Invalid Dataset File Name or Invalid Class Name data.{name}.{camel_name}'
            'Please always name your model file name as `snake_case.py` and class name corresponding `CamelCase`')
    return data_module(**kwargs)
def data_loader(args):
    if  args.train_datatype == 'hdf5':
        alldata = load_data_module(args.dataset,hdf5_dir=args.train_h5path,model='train',aug=True,aug_prob=0.2,args=args)
    else: 
        alldata = load_data_module(args.dataset,image_dir=args.image_dir,msk_dir=args.msk_dir,model='train',aug=True,aug_prob=0.2,args=args)
    trainset,valset=random_split(alldata, [int(len(alldata)*args.split_rate), len(alldata)-int(len(alldata)*args.split_rate)])
    testset=None
    print(len(trainset),len(valset))
    if  args.train_datatype is 'hdf5' and args.test_h5path is not None:
        testset = load_data_module(args.dataset,hdf5_dir=args.test_h5path,model='test',aug=True,aug_prob=0.2,args=args)
        print(len(testset))
    elif args.train_datatype is not 'hdf5' and args.test_img_dir is not None:
        testset = load_data_module(args.dataset,image_dir=args.test_img_dir,msk_dir=args.test_msk_dir,model='test',aug=True,aug_prob=0.2,args=args)
        print(len(testset))
    data_module = DInterface(trainset,valset,testset,**vars(args))
    return trainset,valset,testset,data_module
def main(args):
    pl.seed_everything(args.seed)
    load_path = load_model_path_by_args(args)
    trainset,valset,testset,data_module = data_loader(args)
    if load_path is None:
        model_module = MInterface(**vars(args))
    else:
        model_module = MInterface(**vars(args))
        args.resume_from_checkpoint = load_path

    logger = TensorBoardLogger(save_dir=args.log_dir)
    args.callbacks = load_callbacks(args)
    args.logger = logger
    trainer=Trainer(gpus=1, max_epochs=args.max_epochs,progress_bar_refresh_rate=1,logger=args.logger,callbacks=args.callbacks)
    trainer.fit(model_module,data_module)

    # trainer = Trainer.from_argparse_args(args)
       # from model.unetvggpl import unetvgg
    # model=unetvgg()
    # train_dataloader=DataLoader(trainset, batch_size=args.batch_size,  shuffle=True)
    # val_dataloader=DataLoader(valset, batch_size=args.batch_size,  shuffle=True)
    # trainer.fit(model,train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

In [13]:
main(args)

Global seed set to 42


1000 4000


C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type     | Params
------------------------------------------
0 | train_acc    | Accuracy | 0     
1 | val_acc      | Accuracy | 0     
2 | model_module | unetvgg  | 24.4 M
------------------------------------------
24.4 M    Trainable params
0         Non-t

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Validation sanity check:  50%|█████     | 1/2 [00:04<00:04,  4.37s/it]

C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\segmentation_models_pytorch\base\modules.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


Global seed set to 42
C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\pytorch_lightning\trainer\data_loading.py:433: UserWarning: The number of training samples (17) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Epoch 0:   5%|▍         | 4/84 [00:06<02:01,  1.52s/it, loss=99.9, v_num=15, train_loss=99.90, train_acc=0.000]

C:\yujunchuan\Anaconda\envs\pyt\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
